In [1]:
!pip install facenet_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.

# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls /content/drive/My\ Drive/

# Create Folders for Generated Images

In [3]:
import os

cycled_young = '/content/drive/My Drive/Cycled_Young'
cycled_old = '/content/drive/My Drive/Cycled_Old'

In [ ]:
# Create the folder if necessary
os.makedirs(cycled_young, exist_ok=True)
os.makedirs(cycled_old, exist_ok=True)

# Generators

## Model Definitions

In [4]:
# GENERATOR DEFINITION

import torch
from torch import nn

def up_conv(in_channels, out_channels, kernel_size, stride=1, padding=1,
            scale_factor=2, norm='batch', activ=None):
    """Create a transposed-convolutional layer, with optional normalization."""
    layers = []
    layers.append(nn.Upsample(scale_factor=scale_factor, mode='nearest'))
    layers.append(nn.Conv2d(
        in_channels, out_channels,
        kernel_size, stride, padding, bias=norm is None
    ))
    if norm == 'batch':
        layers.append(nn.BatchNorm2d(out_channels))
    elif norm == 'instance':
        layers.append(nn.InstanceNorm2d(out_channels))

    if activ == 'relu':
        layers.append(nn.ReLU())
    elif activ == 'leaky':
        layers.append(nn.LeakyReLU())
    elif activ == 'tanh':
        layers.append(nn.Tanh())

    return nn.Sequential(*layers)


def conv(in_channels, out_channels, kernel_size, stride=2, padding=1,
         norm='batch', init_zero_weights=False, activ=None, discrim=True):
    """Create a convolutional layer, with optional normalization."""
    layers = []
    conv_layer = nn.Conv2d(
        in_channels=in_channels, out_channels=out_channels,
        kernel_size=kernel_size, stride=stride, padding=padding,
        bias=norm is None
    )
    if init_zero_weights:
        conv_layer.weight.data = 0.001 * torch.randn(
            out_channels, in_channels, kernel_size, kernel_size
        )
    if discrim:
        conv_layer = torch.nn.utils.spectral_norm(conv_layer)
    layers.append(conv_layer)

    if norm == 'batch':
        layers.append(nn.BatchNorm2d(out_channels))
    elif norm == 'instance':
        layers.append(nn.InstanceNorm2d(out_channels))

    if activ == 'relu':
        layers.append(nn.ReLU())
    elif activ == 'leaky':
        layers.append(nn.LeakyReLU())
    elif activ == 'tanh':
        layers.append(nn.Tanh())
    return nn.Sequential(*layers)


class Generator(nn.Module):

    def __init__(self, noise_size, conv_dim=128):
        super().__init__()
        self.up_conv1 = conv(in_channels=noise_size, out_channels=(conv_dim * 4), kernel_size=4, stride=1, padding=3,
                             norm='batch', init_zero_weights=False, activ='relu', discrim=False)
        self.up_conv2 = up_conv(in_channels=(conv_dim * 4), out_channels=(conv_dim * 2), kernel_size=3, stride=1,
                                padding=1, scale_factor=2, norm='batch', activ='relu')
        self.up_conv3 = up_conv(in_channels=(conv_dim * 2), out_channels=conv_dim, kernel_size=3, stride=1, padding=1,
                                scale_factor=2, norm='batch', activ='relu')
        self.up_conv4 = up_conv(in_channels=conv_dim, out_channels=(conv_dim // 2), kernel_size=3, stride=1, padding=1,
                                scale_factor=2, norm='batch', activ='relu')
        self.up_conv5 = up_conv(in_channels=(conv_dim // 2), out_channels=(conv_dim // 4), kernel_size=3, stride=1,
                                padding=1, scale_factor=2, norm='batch', activ='relu')
        self.up_conv6 = up_conv(in_channels=(conv_dim // 4), out_channels=3, kernel_size=3, stride=1,
                                padding=1, scale_factor=2, norm=None, activ='tanh')

    def forward(self, z):
        z = self.up_conv1(z)
        z = self.up_conv2(z)
        z = self.up_conv3(z)
        z = self.up_conv4(z)
        z = self.up_conv5(z)
        z = self.up_conv6(z)
        return z

## Load from State Dict

In [5]:
from facenet_pytorch import InceptionResnetV1
from PIL import Image
from torchvision.utils import save_image
import os
from torchvision import transforms

def add_noise(image_encodings):
    num_samples = image_encodings.shape[0]
    noise_vectors = torch.randn((num_samples, 128), device=device)
    augmented_noise = torch.cat([image_encodings, noise_vectors], dim=1).unsqueeze(2).unsqueeze(3)
    return augmented_noise

# Create generator instances and load the parameters from state dict
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformation for images to be compatible with the generators
transform = transforms.Compose([
                transforms.Resize((128, 128), Image.BICUBIC),
                transforms.CenterCrop((128, 128)),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])

# Face encoder
face_encoder = InceptionResnetV1(pretrained="vggface2").to(device).eval()

# Load the generators
generator_young_to_old = Generator(noise_size=640).to(device)
generator_young_to_old.load_state_dict(torch.load("/content/drive/My Drive/Generator_Weights/g_yto.pth"))
generator_old_to_young = Generator(noise_size=640).to(device)
generator_old_to_young.load_state_dict(torch.load("/content/drive/My Drive/Generator_Weights/g_oty.pth"))

  0%|          | 0.00/107M [00:00<?, ?B/s]

<All keys matched successfully>

# Cycle for Young Images

In [9]:
# YOUNG IMAGES
import numpy as np

young_folder = "/content/drive/My Drive/young_folder"
total_young_count = 0

for img_name in os.listdir(young_folder):

    # Path to image
    img_path = os.path.join(young_folder, img_name)

    # APPLY CYCLE AND SAVE BACK
    young_img = Image.open(img_path).convert("RGB") # Load image
    young_img = transform(young_img).unsqueeze(0).to(device) # Transform the image for model
    young_encoding = add_noise(face_encoder(young_img)) # Encoding
    young_to_old = generator_young_to_old(young_encoding) # Young to old
    re_young = generator_old_to_young(add_noise(face_encoder(young_to_old))) # Cycled back to young
    re_young_img = (re_young[0] + 1) / 2
    re_young_path = os.path.join(cycled_young, img_name)
    save_image(re_young_img, re_young_path) # Save the cycled image to cycled folder

    total_young_count += 1
    print("Current young count:", total_young_count)

Streaming output truncated to the last 5000 lines.
Current young count: 2345
Current young count: 2346
Current young count: 2347
Current young count: 2348
Current young count: 2349
Current young count: 2350
Current young count: 2351
Current young count: 2352
Current young count: 2353
Current young count: 2354
Current young count: 2355
Current young count: 2356
Current young count: 2357
Current young count: 2358
Current young count: 2359
Current young count: 2360
Current young count: 2361
Current young count: 2362
Current young count: 2363
Current young count: 2364
Current young count: 2365
Current young count: 2366
Current young count: 2367
Current young count: 2368
Current young count: 2369
Current young count: 2370
Current young count: 2371
Current young count: 2372
Current young count: 2373
Current young count: 2374
Current young count: 2375
Current young count: 2376
Current young count: 2377
Current young count: 2378
Current young count: 2379
Current young count: 2380
Current young

# Cycle for Old Images

In [10]:
# OLD IMAGES

old_folder = "/content/drive/My Drive/old_folder"
total_old_count = 0

for img_name in os.listdir(old_folder):

    # Path to image
    img_path = os.path.join(old_folder, img_name)

    # APPLY CYCLE AND SAVE BACK
    old_img = Image.open(img_path).convert("RGB") # Load image
    old_img = transform(old_img).unsqueeze(0).to(device) # Transform the image for model
    old_encoding = add_noise(face_encoder(old_img)) # Encoding
    old_to_young = generator_old_to_young(old_encoding) # Old to young
    re_old = generator_young_to_old(add_noise(face_encoder(old_to_young))) # Cycled back to old
    re_old_img = (re_old[0] + 1) / 2
    re_old_path = os.path.join(cycled_old, img_name)
    save_image(re_old_img, re_old_path) # Save the cycled image to cycled folder

    total_old_count += 1
    print("Current old count:", total_old_count)

Current old count: 1
Current old count: 2
Current old count: 3
Current old count: 4
Current old count: 5
Current old count: 6
Current old count: 7
Current old count: 8
Current old count: 9
Current old count: 10
Current old count: 11
Current old count: 12
Current old count: 13
Current old count: 14
Current old count: 15
Current old count: 16
Current old count: 17
Current old count: 18
Current old count: 19
Current old count: 20
Current old count: 21
Current old count: 22
Current old count: 23
Current old count: 24
Current old count: 25
Current old count: 26
Current old count: 27
Current old count: 28
Current old count: 29
Current old count: 30
Current old count: 31
Current old count: 32
Current old count: 33
Current old count: 34
Current old count: 35
Current old count: 36
Current old count: 37
Current old count: 38
Current old count: 39
Current old count: 40
Current old count: 41
Current old count: 42
Current old count: 43
Current old count: 44
Current old count: 45
Current old count: 